# Book Recommend System

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
import os

tf.config.run_functions_eagerly(True)

In [2]:
data = pd.read_csv('Books.csv')

C:\Users\gosfl\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [4]:
data1 = pd.read_csv('Ratings.csv')

In [5]:
data1['Book-Rating'].unique()

array([ 0,  5,  3,  6,  8,  7, 10,  9,  4,  1,  2], dtype=int64)

In [6]:
data1['prefer'] = (data1['Book-Rating']>5).astype(int)

In [7]:
(
    data1
    .groupby('User-ID')
    .agg('count')
    .describe()
)

,ISBN,Book-Rating,prefer
count,105283.000000,105283.000000,105283.000000
mean,10.920851,10.920851,10.920851
std,90.562825,90.562825,90.562825
min,1.000000,1.000000,1.000000
25%,1.000000,1.000000,1.000000
50%,1.000000,1.000000,1.000000
75%,4.000000,4.000000,4.000000
max,13602.000000,13602.000000,13602.000000


In [8]:
num_of_user = len(data1['User-ID'].unique())
num_of_item = len(data1['ISBN'].unique())

num_of_user, num_of_item

(105283, 340556)

In [9]:
def compile_model(num_of_user, num_of_item, learning_rate=0.001): 
    VECTOR_DIMENSION = 8
    
    user_input = keras.layers.Input((1,))
    user_embedding = keras.layers.Embedding(num_of_user, VECTOR_DIMENSION, input_length=1, name='user_embedding')
    user = user_embedding(user_input)
    user = keras.layers.Reshape((VECTOR_DIMENSION, 1))(user)
    
    item_input = keras.layers.Input((1,))
    item_embedding = keras.layers.Embedding(num_of_item, VECTOR_DIMENSION, input_length=1, name='item_embedding')
    item = item_embedding(item_input)
    item = keras.layers.Reshape((VECTOR_DIMENSION, 1))(item)
    
    dot_product = keras.layers.dot([user, item], axes=1) 
    output = keras.layers.Activation('sigmoid')(dot_product)
    
    model = keras.models.Model(inputs=[user_input, item_input], outputs=output) 
    model.compile(
        loss='binary_crossentropy', 
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate), 
        metrics=['binary_accuracy'],
    )
    return model


model = compile_model(num_of_user, num_of_item)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 user_embedding (Embedding)     (None, 1, 8)         842264      ['input_1[0][0]']                
                                                                                                  
 item_embedding (Embedding)     (None, 1, 8)         2724448     ['input_2[0][0]']                
                                                                                              

In [10]:
keras.utils.plot_model(
    model, to_file='movie.png', show_shapes=True, show_layer_activations=True,
)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.


In [11]:
uid2idx = {v: k for k, v in dict(enumerate(data1['User-ID'].unique())).items()}
iid2idx = {v: k for k, v in dict(enumerate(data1['ISBN'].unique())).items()}

dataset = pd.DataFrame()
dataset['uidx'] = data1['User-ID'].map(uid2idx.get)
dataset['iidx'] = data1['ISBN'].map(iid2idx.get)
dataset['y'] = data1['prefer']
dataset

,uidx,iidx,y
0,0,0,0
1,1,1,0
2,2,2,0
3,3,3,0
4,3,4,1
...,...,...,...
1149775,105278,226347,1
1149776,105279,7295,0
1149777,105280,12065,1
1149778,105281,78598,1


In [12]:
data_x = np.array([
    dataset['uidx'].values, 
    dataset['iidx'].values, 
]).T
data_y = np.array([
    dataset['y'].values, 
]).T
print(data_x.shape, data_y.shape)
train_x, valid_x, train_y, valid_y = train_test_split(
    data_x, data_y, test_size=0.1, shuffle = True
)
valid_x, test_x, valid_y, test_y = train_test_split(
    valid_x, valid_y, test_size=0.5, shuffle = True
)
print(
    train_x.shape, train_y.shape, 
    valid_x.shape, valid_y.shape, 
    test_x.shape, test_y.shape,
)

(1149780, 2) (1149780, 1)
(1034802, 2) (1034802, 1) (57489, 2) (57489, 1) (57489, 2) (57489, 1)


In [13]:
checkpoint_filepath = './model_{epoch:02d}_{val_loss:.2f}.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
)

In [ ]:
model = compile_model(num_of_user, num_of_item)
history = model.fit(
    x=(train_x[:,0,None], train_x[:,1,None]),
    y=train_y, 
    batch_size=1024,
    epochs=6, 
    verbose='auto',
    validation_data=((valid_x[:,0,None], valid_x[:,1,None]), valid_y), 
    shuffle=True,
    callbacks=[model_checkpoint_callback],
)

C:\Users\gosfl\anaconda3\lib\site-packages\tensorflow\python\data\ops\structured_function.py:264: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/6
1011/1011 [==============================] - 160s 158ms/step - loss: 0.6930 - binary_accuracy: 0.5147 - val_loss: 0.6918 - val_binary_accuracy: 0.5727
Epoch 2/6
 251/1011 [======>.......................] - ETA: 1:59 - loss: 0.6866 - binary_accuracy: 0.8044

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
best_model = keras.models.load_model('./model_03_0.60.hdf5')

In [ ]:
best_model.evaluate(
    x=(test_x[:,0,None], test_x[:,1,None]),
    y=test_y
)

user_layer = best_model.layers[2]
item_layer = best_model.layers[3]
user_vectors = user_layer.get_weights()[0]
item_vectors = item_layer.get_weights()[0]
user_vectors.shape, item_vectors.shape

In [ ]:
source_df = (
    data1
    .assign(
        bookIdx=lambda df: df['ISBN'].map(iid2idx.get),
    )
    .join(
        data1
        .groupby('ISBN')
        .agg(count=('prefer', 'count'))
        .sort_values('count', ascending=False),
        on='ISBN',
        how='inner',
    )
)
print(len(source_df))
source_df.sort_values('count', ascending=False).head(17)

In [ ]:
data = data.rename(columns={'Book-Title':'bookname'})

In [ ]:
def consine_similarity(v, mat, topn=10):
    sim = np.dot(v, mat.T) / (np.linalg.norm(v) * np.linalg.norm(mat, axis=1))
    ind = np.argpartition(sim, -(topn + 1))[-(topn + 1):]
    ind = ind[np.argsort(sim[ind])][topn - 1::-1]
    return sim[ind], ind

In [ ]:
sim, ind = consine_similarity(item_vectors[113], item_vectors)
print(sim)
print(ind)
source_df[source_df['bookIdx'].isin(ind)]

In [ ]:
def recommend(bookname):
    book_list_name = []
    li = data[data['bookname'] == bookname]
    book_id  = data[data['bookname'] == bookname].index
    book_id = book_id[0]
    
    for newid in islist[book_id]:
        book_list_name.append(data.loc[newid].bookname)
        
        
    return book_list_name   

In [ ]:
recommend('Clara Callan')